<a href="https://colab.research.google.com/github/madhavadama/tensorflow/blob/master/Classification_MNIST_DNN_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST Classification with DNN

In [0]:
import tensorflow as tf

In [0]:
#Reset tensorflow graph
tf.reset_default_graph()

Step 1 : Collect Data

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnists = input_data.read_data_sets('MNIST_data')
trai = mnists.train.images
trt = mnists.train.labels
trt[0:1]

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


array([7], dtype=uint8)

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fc01c34ee10>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fbfd6af3898>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fbfd6af32e8>)

Step 1a : Get Training and Test Data

In [0]:
trainX = mnist.train.images
trainY = mnist.train.labels

In [0]:
testX = mnist.test.images
testY = mnist.test.labels

How many Training and Test Examples?

In [0]:
trainY[0:1]

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])

Lets define some parameters

In [0]:
logs_path = "/content/"
learning_rate = 0.04
n_features = trainX.shape[1]
n_classes = trainY.shape[1]
model_name = 'mnist.ckpt'

#How many examples to feed for training at one time
batch_size = 100

#How many times all the data to be shown
training_epochs = 100

K = 200
L = 100
M = 60
N = 30

# Build the Graph

Input placeholders

In [0]:
with tf.name_scope('input'):
    # None -> batch size can be any size, with n_features
    x = tf.placeholder(tf.float32, shape=[None, n_features], name="x-input") 
    # target n_classes output classes
    y_ = tf.placeholder(tf.float32, shape=[None, n_classes], name="y-input")

Layer 1

In [0]:
with tf.name_scope('layer_1'):
    W1 = tf.Variable(tf.truncated_normal([n_features, K], stddev=0.1))
    b1 = tf.Variable(tf.zeros([K]))
    Y1 = tf.nn.sigmoid(tf.add(tf.matmul(x,W1),b1))

In [0]:
with tf.name_scope('layer_2'):
    W2 = tf.Variable(tf.truncated_normal([K, L], stddev=0.1))
    b2 = tf.Variable(tf.zeros([L]))
    Y2 = tf.nn.sigmoid(tf.add(tf.matmul(Y1,W2),b2))

In [0]:
with tf.name_scope('layer_3'):
    W3 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
    b3 = tf.Variable(tf.zeros([M]))
    Y3 = tf.nn.sigmoid(tf.add(tf.matmul(Y2,W3),b3))

In [0]:
with tf.name_scope('layer_4'):
    W4 = tf.Variable(tf.truncated_normal([M, N], stddev=0.1))
    b4 = tf.Variable(tf.zeros([N]))
    Y4 = tf.nn.sigmoid(tf.add(tf.matmul(Y3,W4),b4))

Prediction    

In [0]:
with tf.name_scope("Output"):
    W5 = tf.Variable(tf.truncated_normal([N,n_classes], stddev=0.1))
    b5 = tf.Variable(tf.zeros([n_classes]))
    y = tf.nn.softmax(tf.matmul(Y4,W5) + b5)

Loss

In [0]:
with tf.name_scope('Loss'):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

GradientDescent Optimizer

In [0]:
with tf.name_scope('train'):        
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)    

Model Accuracy

In [0]:
with tf.name_scope('Accuracy'):
    prediction = tf.argmax(y,1,name="Predict")    
    correct_prediction = tf.equal(prediction, tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32),name="accuracy")

Loss and Accuracy Logging

In [0]:
# create a summary for our cost and accuracy
training_loss = tf.summary.scalar("training_loss", cross_entropy)
training_accuracy = tf.summary.scalar("training_accuracy", accuracy)
test_loss = tf.summary.scalar("test_loss", cross_entropy)
test_accuracy = tf.summary.scalar("test_accuracy", accuracy)

In [0]:
#Create a Saver to save the graph
saver = tf.train.Saver()

# Execute the Graph

In [0]:
#Start Graph execution
with tf.Session() as sess:
    # variables need to be initialized before we can use them
    sess.run(tf.global_variables_initializer())

    # create log writer object
    writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # perform training cycles
    for epoch in range(training_epochs):
        
        # number of batches in one epoch
        batch_count = int(trainX.shape[0]/batch_size)
        
        for i in range(batch_count):
            batch_x  = trainX[i*batch_size:i*batch_size+batch_size]
            batch_y  = trainY[i*batch_size:i*batch_size+batch_size]

            # perform the operations we defined earlier on batch
            _,acc,loss = sess.run([train_op, training_accuracy,training_loss], 
                                  feed_dict={x: batch_x, y_: batch_y})
            
            #log training accuracy and loss
            writer.add_summary(acc, epoch * batch_count + i)
            writer.add_summary(loss, epoch * batch_count + i)    
                       
        #Test loss and accuracy
        #pls note we are giving test data
        acc,loss = sess.run([test_accuracy,test_loss],
                                   feed_dict={x: testX, y_: testY})
        writer.add_summary(acc, epoch * batch_count + i)
        writer.add_summary(loss, epoch * batch_count + i)
        
        if epoch % 5 == 0: 
            print ("Epoch: ", epoch)
            print ("Test Accuracy: ", accuracy.eval(feed_dict={x: testX, y_: testY}))               
    
    
    #Save the model
    saver.save(sess, logs_path + '/' + model_name)

Epoch:  0
Test Accuracy:  0.1028
Epoch:  5
Test Accuracy:  0.1028
Epoch:  10
Test Accuracy:  0.1028
Epoch:  15
Test Accuracy:  0.1852
Epoch:  20
Test Accuracy:  0.2786
Epoch:  25
Test Accuracy:  0.4377
Epoch:  30
Test Accuracy:  0.6776
Epoch:  35
Test Accuracy:  0.7972
Epoch:  40
Test Accuracy:  0.8583
Epoch:  45
Test Accuracy:  0.8895
Epoch:  50
Test Accuracy:  0.9059
Epoch:  55
Test Accuracy:  0.9165
Epoch:  60
Test Accuracy:  0.926
Epoch:  65
Test Accuracy:  0.9319
Epoch:  70
Test Accuracy:  0.9359
Epoch:  75
Test Accuracy:  0.9399
Epoch:  80
Test Accuracy:  0.944
Epoch:  85
Test Accuracy:  0.9467
Epoch:  90
Test Accuracy:  0.9498
Epoch:  95
Test Accuracy:  0.952
